In [408]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [409]:
df =  pd.read_excel('data/output/clean_labeled_data.xlsx')

Raghava

## Classifier

In [410]:
df.head()

,COMMENT,LABEL
0,"thats, bull, say, since, pessimisticvalue, k, ...",NEG
1,"hello, guy, anyone, use, ig, market, issue, pl...",NEU
2,"year, old, hand, pin, falcon, wing, door, subm...",NEU
3,"time, breka, never, really, guys",NEG
4,"world, economy, th, final, one, trump, bankruptcy",NEG


In [411]:
df['LABEL']=df['LABEL'].map({'NEU':'NEG','POS':'POS','NEG':'NEG'})

In [412]:
def classifier(clf,strat,vec):
    
    y = df['LABEL'].astype(str)
    X = df.COMMENT.astype(str)
    #y = pd.get_dummies(y)
    
    if vec == "cv":
        count = CountVectorizer(binary=True,ngram_range=(1,1))
    elif vec == "tdidf":
        count = TfidfVectorizer(binary=True, ngram_range=(1,1), use_idf=False)
    
    if strat == "y":
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        X_train_counts = count.fit_transform(X_train)
        X_test_counts = count.transform(X_test)

        
    elif strat == "n":
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 
        X_train_counts = count.fit_transform(X_train)
        X_test_counts = count.transform(X_test)

    
    if clf == "lr":
        clf = LogisticRegression(solver="newton-cg",C=1).fit(X_train_counts, y_train)
        #clf = LogisticRegression(C=0.4).fit(X_train_counts, y_train)
        
    elif clf == "mnb":
        clf = MultinomialNB().fit(X_train_counts, y_train)
        
    
    elif clf == "svm":
        clf = svm.SVC(kernel='linear').fit(X_train_counts, y_train)

        
    yPred = clf.predict(X_test_counts)
    
    list_ = list(zip(yPred, X_test))
    
    acc=accuracy_score(y_test,yPred)
    f1=f1_score(y_test,yPred, average='weighted')

    return clf, count,"Accuracy: %s" % acc, "F1 %s" % f1, list_


In [413]:
result=classifier("lr","y","cv")
print(result[2:4])

df_result = pd.DataFrame(result[4], columns = ['Predicted_label', 'Comment'])

('Accuracy: 0.745', 'F1 0.712148349698332')


C:\Users\cfo19\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Trying Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([ ('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     'clf', MultinomialNB()),
                    ])

In [414]:
df_result

,Predicted_label,Comment
0,NEG,"hit, pessimisticvalue, morning, go, easter, al..."
1,NEG,bottom
2,NEG,"time, announcement, expect"
3,NEG,"yes, indeed, look, pessimisticvalue, percent, ..."
4,POS,"btc, attempt, stay, daily, however, pull, back..."
5,NEG,"today, point"
6,NEG,"please, dont, panic, there, tesla, space"
7,NEG,"short, folk, agree, overvalue, market, however..."
8,NEG,"wait, next, tweet, mass, destruction, potus"
9,NEG,"market, investor, lose, confidence, stock, pes..."


In [415]:
feature_to_coef = {
    word: coef for word, coef in zip(
        result[1].get_feature_names(), result[0].coef_[0]
    )    
}

In [416]:
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    )[:10]:
    print (best_positive)

('optimisticvalue', -2.422512550325552)
('buy', -1.5710786811484896)
('rise', -1.2797122289554046)
('hold', -0.9872163915830383)
('close', -0.9345578983426864)
('party', -0.9259176500457456)
('firework', -0.912427022694593)
('ok', -0.9071106345044886)
('youd', -0.8893736546303984)
('growth', -0.8739392312184157)


In [417]:
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], reverse=True 
    )[:10]:
    print (best_negative)

('know', 1.4358593353276883)
('sell', 1.3435594110826363)
('need', 1.1392660880089616)
('yet', 1.019535854264432)
('real', 0.9222283185665422)
('low', 0.8437433837231554)
('please', 0.834576609359245)
('look', 0.8150168521096357)
('try', 0.8127270312043091)
('one', 0.7991813499821042)


In [ ]:
def print_top10(vectorizer, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer.get_feature_names()
    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.coef_[i])[-10:]
        print("%s: %s" % (class_label,
              " ".join(feature_names[j] for j in top10)))